In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### I. Load Data

In [10]:
import os
import pandas as pd

project_root = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
data_path = os.path.join(project_root, 'data')

users_path = os.path.join(data_path, 'users.dat')
movies_path = os.path.join(data_path, 'movies.dat')
ratings_path = os.path.join(data_path, 'ratings.dat')

users = pd.read_csv(users_path, sep='::', header=None, engine='python', encoding='ISO-8859-1')
movies = pd.read_csv(movies_path, sep='::', header=None, engine='python', encoding='ISO-8859-1')
ratings = pd.read_csv(ratings_path, sep='::', header=None, engine='python', encoding='ISO-8859-1')

In [11]:
print(users.head())

   0  1   2   3      4
0  1  F   1  10  48067
1  2  M  56  16  70072
2  3  M  25  15  55117
3  4  M  45   7  02460
4  5  M  25  20  55455


In [12]:
print(movies.head())

   0                                   1                             2
0  1                    Toy Story (1995)   Animation|Children's|Comedy
1  2                      Jumanji (1995)  Adventure|Children's|Fantasy
2  3             Grumpier Old Men (1995)                Comedy|Romance
3  4            Waiting to Exhale (1995)                  Comedy|Drama
4  5  Father of the Bride Part II (1995)                        Comedy


In [13]:
print(ratings.head())

   0     1  2          3
0  1  1193  5  978300760
1  1   661  3  978302109
2  1   914  3  978301968
3  1  3408  4  978300275
4  1  2355  5  978824291
